In [1]:
from argparse import Namespace
from pathlib import Path
from glob import glob
import json

scenes = []
for path in glob("../output/*"):
    namespace = eval((Path(path) / "cfg_args").read_text())
    results = None
    if (Path(path) / "results.json").exists():
        results = json.loads((Path(path) / "results.json").read_text())
        scenes.append((namespace,results))

In [2]:
import pandas as pd

results_df = []
for namespace, results in scenes:
    if namespace.source_path.split("/")[-1]=="chair_background":
        for name,res in results.items():
            if "per" not in name:
                results_df.append({
                    "train_run": namespace.model_path.split("/")[-1],
                    "train_mode": "depth" if "depth" in namespace.model_path else "normal",
                    "render_mode": "texture" if "texture" in name else "normal",
                    "method_name":"_".join(name.split("_")[:-1]),
                    "iteration":int(name.split("_")[-1]),
                    **res
                })


results_df = pd.DataFrame(results_df)

In [6]:
results_df = results_df.sort_values("num_gaussians")
results_df["run"] = results_df["train_mode"] + results_df["render_mode"]
is_texture = (results_df["render_mode"]=="texture")
results_df["memory_needed"] = results_df["num_gaussians"]*(48*(~is_texture)+11)*4 + 300000*50*is_texture

In [7]:
import plotly.express as px

def pareto_line(df,x,y,color,fn="max"):
    df = df.copy()
    if fn=="max":
        df = df[df[y]==df.groupby(color)[y].cummax()]
    else:
        df = df[df[y]==df.groupby(color)[y].cummin()]
    
    return px.line(df, x=x, y=y, color=color, log_x=True)

df = results_df[(results_df["run"]=="normalnormal") | (results_df["run"]=="depthtexture")]
fig = pareto_line(df,"num_gaussians","PSNR","run")
fig.show()
fig = pareto_line(df,"num_gaussians","LPIPS","run",fn="min")
fig.show()
fig = pareto_line(df,"num_gaussians","SSIM","run")
fig.show()

In [8]:
pareto_line(df,"memory_needed","PSNR","run").show()
pareto_line(df,"memory_needed","LPIPS","run",fn="min").show()
pareto_line(df,"memory_needed","SSIM","run").show()